In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/team/dataset.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: dataset/storets_CJxxLVJB3G7_0.jpg  
  inflating: __MACOSX/dataset/._storets_CJxxLVJB3G7_0.jpg  
  inflating: dataset/moodndress_CRTB0Pwjm_v_0.jpg  
  inflating: __MACOSX/dataset/._moodndress_CRTB0Pwjm_v_0.jpg  
  inflating: dataset/love_boki_CQga3RaqaOn_0.jpg  
  inflating: __MACOSX/dataset/._love_boki_CQga3RaqaOn_0.jpg  
  inflating: dataset/stylishlytaylored_CJGkw14pFuC_0.jpg  
  inflating: __MACOSX/dataset/._stylishlytaylored_CJGkw14pFuC_0.jpg  
  inflating: dataset/barbaraaleks_CQYMSGRh3D5_0.jpg  
  inflating: __MACOSX/dataset/._barbaraaleks_CQYMSGRh3D5_0.jpg  
  inflating: dataset/maybebaby.official_CPz-pN9nZ6y_0.jpg  
  inflating: __MACOSX/dataset/._maybebaby.official_CPz-pN9nZ6y_0.jpg  
  inflating: dataset/storets_CLVV4D2BvuE_0.jpg  
  inflating: __MACOSX/dataset/._storets_CLVV4D2BvuE_0.jpg  
  inflating: dataset/livia_auer_CN9tr3IiDAo_0.jpg  
  inflating: __MACOSX/dataset/._livia_auer_CN9tr3IiDAo_0.jpg  
  inflating: dataset/aac

In [3]:
#Library Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from google.colab import output
import torch.optim as optim
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
df = pd.read_csv("/content/drive/MyDrive/team/isImage.csv",lineterminator='\n')
img_path = '/content/dataset/'

In [5]:
  df

,Unnamed: 0,image_name,isImage
0,0,kristina.kraemer_B6x_f_mIfpl_0.jpg,1
1,1,aug_min_B6zSJWPltbf_0.jpg,1
2,2,statecashmere_B60mnfVntEW_0.jpg,0
3,3,statecashmere_B63ejyLFy6Z_0.jpg,1
4,4,aug_min_B62unlNld81_0.jpg,0
...,...,...,...
44818,44818,commonunique_lookbook_CRiJsSiNdrt_0.jpg,1
44819,44819,wconceptkorea_CRh6JGNlDYL_0.jpg,1
44820,44820,labauhinia_CRinhv3MCqk_0.jpg,1
44821,44821,storets_CRh0aY6hQFA_0.jpg,1


In [6]:
class ToTensor(object):
    """numpy array를 tensor(torch)로 변환합니다."""
    def __call__(self, sample):
        image, label = sample['image_name'], sample['isImage']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        #plt.imshow(image)
        #plt.show()
        image = image.transpose((2, 0, 1))
        return {'image_name': torch.FloatTensor(image),
                'isImage': torch.FloatTensor(label)}
# to_tensor 선언
to_tensor = T.Compose([
                        ToTensor()
                    ])

class Dataset(torch.utils.data.Dataset):
    def __init__(self,
                 dir_path,
                 meta_df,
                 transforms=to_tensor,#미리 선언한 to_tensor를 transforms로 받음
                 augmentations=None):
        
        self.dir_path = dir_path # 데이터의 이미지가 저장된 디렉터리 경로
        self.meta_df = meta_df # 데이터의 인덱스와 정답지가 들어있는 DataFrame

        self.transforms = transforms# Transform
        self.augmentations = augmentations # Augmentation
        
    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, index):
        image = cv2.imread(self.dir_path + self.meta_df['image_name'].iloc[index])
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = cv2.resize(image,(224,224))
        
        #image = (image/255).astype('float')[..., np.newaxis] 
        #image = torch.from_numpy(image).float()

        # 정답 numpy array생성(존재하면 1 없으면 0)
        label = self.meta_df['isImage'].iloc[index].astype('float')
        label = np.asarray(label).reshape(1,-1)[0,:]

        sample = {'image_name': image, 'isImage': label}

        # transform 적용
        # numpy to tensor
        if self.transforms:
            sample = self.transforms(sample)

        # sample 반환
        return sample

In [7]:
# nn.Module을 상속 받아 MultiLabelResnet를 정의
class MultiLabelResnet(nn.Module):
    def __init__(self):
        super(MultiLabelResnet, self).__init__()
        self.conv2d = nn.Conv2d(3, 3, 3, stride=1)
        self.resnet = models.resnet18() 
        self.FC = nn.Linear(1000, 1)

    def forward(self, x):
        # resnet의 입력은 [3, N, N]으로
        # 3개의 채널을 갖기 때문에
        # resnet 입력 전에 conv2d를 한 층 추가
        x = F.relu(self.conv2d(x))

        # resnet18을 추가
        x = F.relu(self.resnet(x))

        # 마지막 출력에 nn.Linear를 추가
        # multilabel을 예측해야 하기 때문에
        # softmax가 아닌 sigmoid를 적용
        x = torch.sigmoid(self.FC(x))
        return x
# 모델 선언
model = MultiLabelResnet()

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

# dirty_mnist_answer에서 train_idx와 val_idx를 생성
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(df),1):
    print(f'[fold: {fold_index}]')
    
    # cuda cache 초기화
    torch.cuda.empty_cache()

    #train fold, validation fold 분할
    train_answer = df.iloc[trn_idx]
    valid_answer  = df.iloc[val_idx]

    #Dataset 정의
    train_dataset = Dataset('/content/dataset/', train_answer)
    valid_dataset = Dataset('/content/dataset/', valid_answer)

    #DataLoader 정의
    train_data_loader = DataLoader(
        train_dataset,
        batch_size = 16,
        shuffle = False,
        num_workers = 2
    )
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size = 8,
        shuffle = False,
        num_workers = 2
    )

    # 모델 선언
    model = MultiLabelResnet()
    model.to(device)# gpu에 모델 할당

    # 훈련 옵션 설정
    optimizer = torch.optim.Adam(model.parameters(),
                                lr = 0.001)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size = 5,
                                                gamma = 0.75)
    criterion = torch.nn.BCELoss()

    # 훈련 시작
    valid_acc_max = 0
    for epoch in range(10):
        # 1개 epoch 훈련
        train_acc_list = []
        with tqdm(train_data_loader,#train_data_loader를 iterative하게 반환
                total=train_data_loader.__len__(), # train_data_loader의 크기
                unit="batch") as train_bar:# 한번 반환하는 smaple의 단위는 "batch"
            for sample in train_bar:
                train_bar.set_description(f"Train Epoch {epoch}")
                # 갱신할 변수들에 대한 모든 변화도를 0으로 초기화
                # 참고)https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html
                optimizer.zero_grad()
                images, labels = sample['image_name'], sample['isImage']
                # tensor를 gpu에 올리기 
                images = images.to(device)
                labels = labels.to(device)
            
                # 모델의 dropoupt, batchnormalization를 train 모드로 설정
                model.train()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.set_grad_enabled(True):
                    # 모델 예측
                    probs  = model(images)
                    # loss 계산
                    loss = criterion(probs, labels)
                    # 중간 노드의 gradient로
                    # backpropagation을 적용하여
                    # gradient 계산
                    loss.backward()
                    # weight 갱신
                    optimizer.step()

                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
                    train_acc_list.append(batch_acc)
                    train_acc = np.mean(train_acc_list)

                # 현재 progress bar에 현재 미니배치의 loss 결과 출력
                train_bar.set_postfix(train_loss= loss.item(),
                                      train_acc = train_acc)
    
     
        # 1개 epoch학습 후 Validation 점수 계산
        valid_acc_list = []
        with tqdm(valid_data_loader,
                total=valid_data_loader.__len__(),
                unit="batch") as valid_bar:
                
                for sample in valid_bar:
                  valid_bar.set_description(f"Valid Epoch {epoch}")
                  optimizer.zero_grad()
                  images, labels = sample['image_name'], sample['isImage']
                  images = images.to(device)
                  labels = labels.to(device)

                  # 모델의 dropoupt, batchnormalization를 eval모드로 설정
                  model.eval()
                  # .forward()에서 중간 노드의 gradient를 계산
                  with torch.no_grad():
                      # validation loss만을 계산
                      probs  = model(images)
                      valid_loss = criterion(probs, labels)

                      # train accuracy 계산
                      probs  = probs.cpu().detach().numpy()
                      labels = labels.cpu().detach().numpy()
                      preds = probs > 0.5
                      batch_acc = (labels == preds).mean()
                      valid_acc_list.append(batch_acc)

                  valid_acc = np.mean(valid_acc_list)
                  valid_bar.set_postfix(valid_loss = valid_loss.item(),
                                        valid_acc = valid_acc)
                
                  # Learning rate 조절
                  lr_scheduler.step()

                  # 모델 저장
                  if valid_acc_max < valid_acc:
                      valid_acc_max = valid_acc
                      best_model = model
                      MODEL = "resnet18"
                      # 모델을 저장할 구글 드라이브 경로
                      path = "/content/drive/MyDrive/team/models/"
                      torch.save(best_model, f'{path}{fold_index}_{MODEL}_{valid_loss.item():2.4f}_epoch_{epoch}.pth') 
                
    # 폴드별로 가장 좋은 모델 저장
    best_models.append(best_model)

[fold: 1]


Valid Epoch 9: 100%|██████████| 1121/1121 [04:07<00:00,  4.53batch/s, valid_acc=0.855, valid_loss=0.098]


[fold: 2]


Valid Epoch 6:  77%|███████▋  | 866/1121 [03:14<00:59,  4.30batch/s, valid_acc=0.843, valid_loss=0.382]